In [ ]:
import pandas as pd

In [ ]:
import json

with open("../data/processed/topics2021.jsonl", encoding="utf8") as f:
    patients = pd.DataFrame(json.loads(line) for line in f)

In [ ]:
patients['nn'] = patients['negated_entities'].str.len()
patients['na'] = patients['positive_entities'].str.len()
patients['nf'] = patients['fh_entities'].str.len()
patients['np'] = patients['pmh_entities'].str.len()

In [ ]:
patients.head()

In [ ]:
topics = pd.read_csv("../../clinical-trials/ecir-2023-paper/topcis_clustering.csv")
topics.head()

In [ ]:
new = pd.read_csv("../../clinical-trials/ecir-2023-paper/new.csv", names=['metric', 'query', 'score'])
baseline = pd.read_csv("../../clinical-trials/ecir-2023-paper/baseline.csv", names=['metric', 'query', 'score'])
baseline.head()

In [ ]:
baseline.index = pd.MultiIndex.from_arrays(baseline[['query', 'metric']].values.T, names=['query', 'metric'])
baseline.head()

new.index = pd.MultiIndex.from_arrays(new[['query', 'metric']].values.T, names=['query', 'metric'])
new.head()

In [ ]:
base_df = baseline.unstack().iloc[:,-4:]
new_df = new.unstack().iloc[:,-4:]

In [ ]:
new_df.head()

In [ ]:
base_df.head()

In [ ]:
new_df.columns = ['P10', 'ndcg10', 'ndcg5', 'rr']
new_df = new_df.reset_index()

base_df.columns = ['P10', 'ndcg10', 'ndcg5', 'rr']
base_df = base_df.reset_index()

In [ ]:
base_df.head()

In [ ]:
new_df['P10'] - base_df['P10']

In [ ]:
import seaborn as sns

In [ ]:
sns.lineplot(new_df['P10'])
sns.lineplot(base_df['P10'])

In [ ]:
sns.lineplot(new_df['rr'])
sns.lineplot(base_df['rr'])

In [ ]:
res_df = pd.DataFrame()
for topic in topics['Topic'].unique().tolist():
    topic_ids = topics[topics['Topic'] == topic]['Disease area or specialty'].tolist()

    print(topic)
    x = new_df[new_df['query'].isin(topic_ids)] -  base_df[base_df['query'].isin(topic_ids)]
    # print(x.mean())
    sss = x.mean()
    sss['topic'] = topic
    sss['na'] = patients[patients['patient_id'].isin(topic_ids)]['na'].mean()
    sss['nn'] = patients[patients['patient_id'].isin(topic_ids)]['nn'].mean()
    sss['np'] = patients[patients['patient_id'].isin(topic_ids)]['np'].mean()
    sss['nf'] = patients[patients['patient_id'].isin(topic_ids)]['nf'].mean()
    sss['total'] = sss['na'] + sss['nn'] + sss['np'] + sss['nf']
    print(sss)


    res_df = res_df.append(sss, ignore_index=True)

In [ ]:
res_df

In [ ]:
res_df.sort_values('rr')

# quantify negations

In [ ]:
diff_df = new_df - base_df
diff_df['query'] = new_df['query']

In [ ]:
diff_df = diff_df.merge(patients, left_on='query', right_on='patient_id')

In [ ]:
diff_df.sort_values('nn').plot.scatter(x='nn', y='P10')

In [ ]:
diff_df.sort_values('nn').plot.scatter(y='na', x='rr')

In [ ]:
diff_df.sort_values('nn').plot.scatter(y='np', x='rr')

In [ ]:
diff_df.sort_values('rr')

In [ ]:
diff_df.sort_values('nn')